# KOBERT를 이용해 ESG 분류하기


`ESG,긍부정 학습 순서`
### 1차 학습(model + 추가 layer : requires_grad = True)
- 1. model 선언하기
- 2. ESG 데이터로더로 1차 학습
- 3. model + 추가 layer 전체 state_dict 저장하기

### 2차 학습(model + esg 추가 layer + 긍부정 layer)

- a. model + esg layer : rg = False, 긍부정 layer : rg = True
- b. model + esg layer + 긍부정 layer  : re = True

- 1. model 선언하기(1차 학습 state_dict 적용)
- 2. 긍부정 데이터로더로 2차 학습
- 3. model,esg,긍부정 layer 모두 state_dict 저장하기


`참고 블로그 모음 `

# 모델링 관련 참고 사이트

### kobert modeling code
- kobert 감정분류 코드 : https://velog.io/@sseq007/Kobert-%EB%AA%A8%EB%8D%B8-%EC%82%AC%EC%9A%A91

### 파라미터 저장 및 재적용 관련 참고 사이트
- pytorch 공식 홈페이지 모델 저장하고 불러오기 : https://tutorials.pytorch.kr/beginner/saving_loading_models.html

### requires_grad = True 일부 계층 적용 관련 참고 사이트
- 개인 블로그 : https://jeonghyeokpark.netlify.app/pytorch/2020/11/27/pytorch1.html

### 모델의 파라미터 접근하기
- 개인 블로그 : https://soundprovider.tistory.com/entry/pytorch-torch%EC%97%90%EC%84%9C-parameter-%EC%A0%91%EA%B7%BC%ED%95%98%EA%B8%B0
- 개인 블로그 : https://dbwp031.tistory.com/25

### 파이토치 기초 정리
- 이수안컴퓨터연구소 파이토치 기초 정리 : https://www.youtube.com/watch?v=k60oT_8lyFw&t=8494s

### 모델의 학습모드와 평가모드 설명
- 개인 블로그 : https://tigris-data-science.tistory.com/entry/PyTorch-modeltrain-vs-modeleval-vs-torchnograd

### 데이터 프레임 loc를 이용한 값 변경
- 개인 블로그 : https://velog.io/@skkumin/Pandas-loc%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84-%EA%B0%92-%EB%B3%80%EA%B2%BD%ED%95%98%EA%B8%B0

### train_test_split 사용방법
- 개인 블로그 : https://smalldatalab.tistory.com/23

### Dataset과 DataLoader, transform 사용방법
- Dataset & DataLoader 파이토치 튜토리얼 : https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html
- transform : 파이토치 튜토리얼 : https://tutorials.pytorch.kr/beginner/basics/transforms_tutorial.html
- 개인 블로그 : https://curiousseed.tistory.com/76

### gluonnlp의 bertsentenceTransform 설명
- gluonnlp 공식 사이트 : https://nlp.gluon.ai/api/modules/data.html#gluonnlp.data.BERTSentenceTransform

### model.zero_grad vs optimizer.zero_grad 차이점 설명
- 개인 블로그 : https://otzslayer.github.io/pytorch/2022/04/17/difference-between-zero-grads.html

### 구글 코랩에서 모델 저장하는 방법
- 개인 블로그 : https://velog.io/@bpbpbp_yosep/PyTorch%EC%97%90%EC%84%9C-%EB%AA%A8%EB%8D%B8-%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0

### 드라이브 파일 구글 코랩에 업로드 하는 방법
- 개인 블로그 : https://rfriend.tistory.com/m/564

`필요 환경 및 패키지 설치`

In [ ]:
!pip install mxnet
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install gluonnlp==0.10.0
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
# 오류 메세지 frames 열고 onp.bool -> bool로 변경하기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import urllib.request

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kobert_tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1') # input을 만드는 tokenize 명령을 할 수 있는 tokenizer

kobertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False) # 코버트 모델 불러오기

vocab = nlp.vocab.BERTVocab.from_sentencepiece(kobert_tokenizer.vocab_file, padding_token='[PAD]') # tokenize의 기반이 되는 사전

`kobert_tokenizer 이해 예시`

In [4]:
word_tokenizer = kobert_tokenizer.tokenize

# kobert_tokenizer(문장) vs kobert_tokenizer.tokenize(문장) 비교하기

print('kobert_tokenizer')

print(kobert_tokenizer('나는 집에 가고 싶다')) # 단어를 분리하고 임베딩까지 한다.

print('kobert_tokenizer.tokenize')

print(word_tokenizer('나는 집에 가고 싶다')) # 단어를 분리한다.

# kobert_tokenizer(문장)이 반환하는 항목 설명

# {
#     input_ids : [스페셜 토큰이 포함된 문장 임베딩]
#     token_type_ids : [문장의 구분을 나누는 문장 임베딩 첫 문장 0]
#     attention_mask : [문장 길이가 다를 때 긴 것에 기준을 맞추고 짧은 것은 패딩 | 패딩 된 부분 : 0 (0이어서 계산에 반영되는 않는 무시의 용도), 패딩 안 된 부분 : 1]
# }

kobert_tokenizer
{'input_ids': [2, 1375, 4384, 6896, 517, 5330, 5439, 3072, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
kobert_tokenizer.tokenize
['▁나는', '▁집', '에', '▁', '가', '고', '▁싶다']


` 구글 드라이브 연동하기`

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# 현재 경로 확인하는 법
!pwd

Mounted at /content/drive
/content


`사전 파라미터 지정`

In [6]:
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

`esg 데이터셋 불러오기 및 dataloader 구축`

In [7]:
# esg 데이터 불러오기
esg_data = pd.read_csv('/content/drive/MyDrive/kobert_modeling/문장esg테스트용.csv')

# 필요없는 정보 제거하기
del esg_data['Unnamed: 0']

# 문장 라벨 묶어주기
esg_data_list = []
for sent, e_label, s_label, g_label in zip(esg_data['문장'], esg_data['E'], esg_data['S'], esg_data['G']):
    data = []
    data.append(str(sent))
    data.append((str(e_label), str(s_label), str(g_label)))

    esg_data_list.append(data)

# train_test_split 활용해서 train_data와 valid_data 구분하기
from sklearn.model_selection import train_test_split
esg_train_data, esg_valid_data = train_test_split(esg_data_list, test_size = 0.2, shuffle = True, random_state = 32)

# kobert 모델에 input으로 들어갈 데이터셋(BERTDataset)을 만들어주는 클래스
class ESG_BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        # Kobert 모델에 들어가기 위한 input을 만드는 함수입니다.

        # dataset: 내 데이터셋 : [문장, 라벨]
        # sent_idx: 0 : 문장 index
        # label_idx: 1 : 라벨 index
        # bert_tokenizer: 사용할 tokenizer : 사전에 정의하고 생성해야 함.
        # max_len: input sentence의 최대 길이
        # pad: (True/False) : max_len을 고려한 패딩 여부 결정
        # pair: (True/False) : input이  [sent a, sent b]처럼 문장쌍으로 들어가는지 여부

        self.transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab = vocab, pad=pad, pair=pair)

        # nlp.data.BERTSentenceTransform() 사용 방법

        # input 정리
        # tokenizer(bert_tokenizer)
        # max_seq_length : 위 참고
        # vocab : CLS, SEP 등등의 토큰이 임베딩 되어 있는 사전
        # pad : 위 참고
        # pair : 위 참고

        # output 정리
        # token_ids : vocab에 따른 임베딩
        # valid_len : 문장의 실제 길이
        # tokekn_type : 문장 pair 여부를 보여주는 0,1로 구성된 positional encoding


        self.sentences = [self.transform([i[sent_idx]]) for i in dataset] # 데이터 셋의 문장들을 bert 맞춤형 토큰화 모음 리스트
        self.labels = torch.tensor([(int(i[label_idx][0]), int(i[label_idx][1]), int(i[label_idx][2])) for i in dataset]) # 데이터 셋의 라벨들 모음 리스트
        # self.labels 형태 예시 : (0,1,0)


    def __getitem__(self, i): # 특정 문장 특정 문장의 라벨을 리턴해줌
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels)) # 문장 개수 리턴 해줌.


esg_data_train = ESG_BERTDataset(esg_train_data, 0, 1, word_tokenizer, vocab, max_len, True, False)
esg_data_test = ESG_BERTDataset(esg_valid_data, 0, 1, word_tokenizer, vocab, max_len, True, False)


# DataLoader의 역할 : 한번에 batch_size만큼 학습시키는(테스트 하는) iterable 생성
esg_train_dataloader = torch.utils.data.DataLoader(esg_data_train, batch_size = batch_size, num_workers = 5)
esg_test_dataloader = torch.utils.data.DataLoader(esg_data_test, batch_size = batch_size, num_workers = 5)

`긍부정 데이터셋 불러오기 및 dataloader 구축`

In [8]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

# 긍정 1 부정 0
pn_train_data = pd.read_table('ratings_train.txt')
pn_test_data = pd.read_table('ratings_test.txt')
# pn_train_data
pn_train_data_list = []
for sent, label in zip(pn_train_data['document'], pn_train_data['label']):
    pn_train_data = []
    pn_train_data.append(str(sent))
    pn_train_data.append(str(label))

    pn_train_data_list.append(pn_train_data)

#pn_test_data
pn_test_data_list = []
for sent, label in zip(pn_test_data['document'], pn_test_data['label']):
    pn_test_data = []
    pn_test_data.append(str(sent))
    pn_test_data.append(str(label))

    pn_test_data_list.append(pn_test_data)

class SENT_BERTDataset(Dataset): # bert 모델에 어떤 데이터셋을 넣을 줄 것인지 결정 : 어떤 데이터셋을 받아와서 어떻게 token화 해서 넣을 거야??? 를 결정한다.
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        self.transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab = vocab, pad=pad, pair=pair)
        self.sentences = [self.transform([i[sent_idx]]) for i in dataset] # 데이터 셋의 문장들을 bert 맞춤형 토큰화 모음 리스트
        self.labels = [np.int32(i[label_idx]) for i in dataset] # 데이터 셋의 라벨들 모음 리스트

    def __getitem__(self, i): # 특정 문장 특정 문장의 라벨을 리턴해줌
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels)) # 문장 개수 리턴 해줌.


pn_data_train = SENT_BERTDataset(pn_train_data_list, 0, 1, word_tokenizer, vocab, max_len, True, False)
pn_data_test = SENT_BERTDataset(pn_test_data_list, 0, 1, word_tokenizer, vocab, max_len, True, False)

# DataLoader의 역할 : 한번에 batch_size만큼 시키는 iterable 생성
pn_train_dataloader = torch.utils.data.DataLoader(pn_data_train, batch_size = batch_size, num_workers = 5)
pn_test_dataloader = torch.utils.data.DataLoader(pn_data_test, batch_size = batch_size, num_workers = 5)

`--------------------------------------모델 구축----------------------------------------`
- 과정1. esg 데이터셋을 esg분류 모델에 넣는다. 이때 esg 추가 계층의 requires_grad만 True로 한다.
- 과정2. 학습을 시킨 후 esg 추가 계층의 state_dict만 저장한다.
- 과정3. 긍부정 데이터셋을 긍부정분류 모델에 넣는다. 이때 esg 추가 계층의 requires_grad는 False 이고 긍부정분류 계층의 requires_grad = True이다.
- 과정4. 학습을 시킨 후 긍부정 추가 계층의 state_dict만 저장한다
- 과정5. 두가지 추가 계층을 모두 가진 모델을 구축하고 2,4에서 저장한 state_dict를 각각의 계층에 적용시킨다.

# ESG를 분류하는 모델

In [ ]:
class BERT_ESG_Classifier(nn.Module):
    def __init__(self,
                 bert, # bert 모델 받아오기
                 hidden_size = 768, # 은닉층의 크기(기준 수)
                 num_classes = 3,   # [E,S,G]
                 dr_rate = None, # dropout_rate : 신경망 학습 중에 일부 뉴런을 무작위로 제거하여 과적합을 방지하고 모델의 일반화 성능을 향상시키는 기법
                 params = None):
        super(BERT_ESG_Classifier, self).__init__()
        self.bert = bert # 사용할 bert 모델 지정
        self.dr_rate = dr_rate # dropout 비율 지정
        self.esg_classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate: # dr_rate가 0과 None이 아니고 (0~1) 사이로 설정되었을 경우에는
            self.dropout = nn.Dropout(p = dr_rate) # nn.dropout을 실행한다.
        # (E,S,G)와의 연관성을 0~1 사이로 보여주는 sigmoid 함수 추가
        self.activation_sigmoid = nn.Sigmoid()

    def gen_attention_mask(self, token_ids, valid_length): # attention mask를 생성하는 것 : 이 문장을 바라보는 전문가들 생성한다.
        attention_mask = torch.zeros_like(token_ids) # token_ids와 같은 크기를 가지고 있는 0으로 채워지는 것들  : 뭔가 포지셔널 인코딩일 것 같은 느낌
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1 # 가리지 않는 부분 설정하기
        return attention_mask.float() # dtype 은 float32로 한다.

    def forward(self, token_ids, valid_length, segment_ids): # segment_ids 는 문장단위를 나누는 임베딩 부분인 것 같다. 한 문장일 경우 0000, 두 문장 이상일 경우 00..111
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        # torch.long() : int64 타입
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        esg_score = self.esg_classifier(out)
        activation_esg_score = self.activation_sigmoid(esg_score)
        return activation_esg_score

# ESG 모델 + 긍부정 분류 모델

In [ ]:
class BERT_SENT_Classifier(nn.Module):
    def __init__(self,
                 bert, # bert 모델 받아오기
                 hidden_size = 768, # 은닉층의 크기(기준 수)
                 num_classes = 3,   # [E,S,G]
                 dr_rate = None, # dropout_rate : 신경망 학습 중에 일부 뉴런을 무작위로 제거하여 과적합을 방지하고 모델의 일반화 성능을 향상시키는 기법
                 params = None):
        super(BERT_ESG_Classifier, self).__init__()
        self.bert = bert # 사용할 bert 모델 지정
        self.dr_rate = dr_rate # dropout 비율 지정
        self.esg_classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate: # dr_rate가 0과 None이 아니고 (0~1) 사이로 설정되었을 경우에는
            self.dropout = nn.Dropout(p = dr_rate) # nn.dropout을 실행한다.
        # (E,S,G)와의 연관성을 0~1 사이로 보여주는 sigmoid 함수 추가
        self.activation_sigmoid = nn.Sigmoid()
        self.sent_clssifier = nn.Linear(num_classes, 2)
        self.activation_softmax = nn.Softmax(dim = 1)

    def gen_attention_mask(self, token_ids, valid_length): # attention mask를 생성하는 것 : 이 문장을 바라보는 전문가들 생성한다.
        attention_mask = torch.zeros_like(token_ids) # token_ids와 같은 크기를 가지고 있는 0으로 채워지는 것들  : 뭔가 포지셔널 인코딩일 것 같은 느낌
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1 # 가리지 않는 부분 설정하기
        return attention_mask.float() # dtype 은 float32로 한다.

    def forward(self, token_ids, valid_length, segment_ids): # segment_ids 는 문장단위를 나누는 임베딩 부분인 것 같다. 한 문장일 경우 0000, 두 문장 이상일 경우 00..111
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        # torch.long() : int64 타입
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        esg_score = self.esg_classifier(out)
        posneg = self.sent_classifier(esg_score)
        activation_sent_score = self.activation_softmax(posneg)
        return activation_sent_score

# 최종 esg/긍부정 모델

In [ ]:
class BERT_ESG_SENT_Classifier(nn.Module):
    def __init__(self,
                 bert, # bert 모델 받아오기
                 hidden_size = 768, # 은닉층의 크기(기준 수)
                 num_classes = 3,   # [E,S,G]
                 dr_rate = None, # dropout_rate : 신경망 학습 중에 일부 뉴런을 무작위로 제거하여 과적합을 방지하고 모델의 일반화 성능을 향상시키는 기법
                 params = None):
        super(BERT_ESG_Classifier, self).__init__()
        self.bert = bert # 사용할 bert 모델 지정
        self.dr_rate = dr_rate # dropout 비율 지정
        self.esg_classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate: # dr_rate가 0과 None이 아니고 (0~1) 사이로 설정되었을 경우에는
            self.dropout = nn.Dropout(p = dr_rate) # nn.dropout을 실행한다.
        # (E,S,G)와의 연관성을 0~1 사이로 보여주는 sigmoid 함수 추가
        self.activation_sigmoid = nn.Sigmoid()
        self.sent_clssifier = nn.Linear(num_classes, 2)
        self.activation_softmax = nn.Softmax(dim = 1)

    def gen_attention_mask(self, token_ids, valid_length): # attention mask를 생성하는 것 : 이 문장을 바라보는 전문가들 생성한다.
        attention_mask = torch.zeros_like(token_ids) # token_ids와 같은 크기를 가지고 있는 0으로 채워지는 것들  : 뭔가 포지셔널 인코딩일 것 같은 느낌
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1 # 가리지 않는 부분 설정하기
        return attention_mask.float() # dtype 은 float32로 한다.

    def forward(self, token_ids, valid_length, segment_ids): # segment_ids 는 문장단위를 나누는 임베딩 부분인 것 같다. 한 문장일 경우 0000, 두 문장 이상일 경우 00..111
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        # torch.long() : int64 타입
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        esg_score = self.esg_classifier(out)
        activation_esg_score = self.activation_sigmoid(esg_score)

        posneg = self.sent_classifier(esg_score)
        activation_sent_score = self.activation_softmax(posneg)
        return activation_esg_score, activation_sent_score

`모델 정의하기`

In [ ]:
# BERT  모델 불러오기
model = BERT_ESG_Classifier(kobertmodel,  dr_rate = 0.5).to(device)

In [ ]:
# freezing 결과 확인하기
for name, para in model.named_parameters() :
    print(para.requires_grad)


`모델에 호환되는 optimizer, loss_function, scheduler 정의`

In [ ]:
# 옵티마이저 생성 시 전달해줄 파라미터 정의
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [ # model.named_parameters()는 model의 해당층이름/ 해당층 가중치를 리턴
    {'params': [weight for name, weight in model.named_parameters() if not any(nd in name for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [weight for name, weight in model.named_parameters() if any(nd in name for nd in no_decay)], 'weight_decay': 0.0} ]


# 옵티마이저 정의 : model로 부터 온 것 : model과 연관
optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)

# 손실함수 정의
loss_fn = nn.BCELoss()

# 스케쥴러 생성 시 전달해줄 파라미터 정의
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

# 스케쥴러 정의 : model로 부터 온 것 : model과 연관
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


`optimizer vs scheduler`
- optimizer : 가중치 업데이트를 어떤 방식으로 할 것인가?
- scheduler : 학습률을 어떤 방식으로 조절할 것인가?

`정확도 계산하는 함수 정의`

In [ ]:
# esg 분류 시에 사용한다
def esg_calc_accuracy(X,Y):
    X[X <= 0.5 ] = 0
    X[X > 0.5 ] = 1
    answer = 0
    for pred in (X - Y) :
        if torch.abs(pred).sum() == 0 :
            answer += 1
    train_acc = answer/batch_size
    return train_acc

# 긍부정 분류 시에 사용한다.
def sent_calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

`모델 학습 코드`

In [ ]:
# esg 분류 시에 사용한다.
for e in range(1):
    train_acc = 0.0
    model.train() # model을 훈련모드로 바꾸고, 가중치가 업데이트 될 수 있게 한다.
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(esg_train_dataloader)):
        # 옵티마이저의 미분값을 0으로 초기화
        optimizer.zero_grad()

        # model의 forward 인자 설정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.float().to(device)

        # model output 도출
        out = model.forward(token_ids, valid_length, segment_ids)

        # 모델 output과 label(정답)과의 손실함수 정의
        loss = loss_fn(out, label)

        # 손실함수의 기울기 계산
        loss.backward()

        # gradient vanishing 또는 gradient exploding을 방지하기 위한 gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 기울기 반영한 가중치 업데이트
        optimizer.step()
        scheduler.step()

        train_acc += esg_calc_accuracy(out, label)

        if batch_id % log_interval == 0:
             print(f'{e+1} 번 반복 | 배치순서 {batch_id + 1} | 오차 정도 {loss.data.cpu().numpy()}| 정확도 {train_acc / (batch_id+1)}')

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

<ipython-input-35-a2bb859b5912>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/951 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


1 번 반복 | 배치순서 1 | 오차 정도 1.612990140914917| 정확도 0.125
1 번 반복 | 배치순서 201 | 오차 정도 1.3686542510986328| 정확도 0.39552238805970147
1 번 반복 | 배치순서 401 | 오차 정도 1.3103528022766113| 정확도 0.427213216957606
1 번 반복 | 배치순서 601 | 오차 정도 1.1976993083953857| 정확도 0.43612208818635606
1 번 반복 | 배치순서 801 | 오차 정도 1.3548005819320679| 정확도 0.4432740324594257
epoch 1 train acc 0.4473416140904311


In [ ]:
# 감정 분류 시에 사용한다.
for e in range(1):
    train_acc = 0.0
    model.train() # model을 훈련모드로 바꾸고, 가중치가 업데이트 될 수 있게 한다.
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(pn_train_dataloader)):
        # 옵티마이저의 미분값을 0으로 초기화
        optimizer.zero_grad()

        # model의 forward 인자 설정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # model output 도출
        out = model.forward(token_ids, valid_length, segment_ids)

        # 모델 output과 label(정답)과의 손실함수 정의
        loss = loss_fn(out, label)

        # 손실함수의 기울기 계산
        loss.backward()

        # gradient vanishing 또는 gradient exploding을 방지하기 위한 gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 기울기 반영한 가중치 업데이트
        optimizer.step()
        scheduler.step()

        train_acc += sent_calc_accuracy(out, label)

        if batch_id % log_interval == 0:
             print(f'{e+1} 번 반복 | 배치순서 {batch_id + 1} | 오차 정도 {loss.data.cpu().numpy()}| 정확도 {train_acc / (batch_id+1)}')

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

`학습 완료시킨 model을 이용해서 test(predict) 해보기`

In [ ]:
# esg 분류시에 사용한다.
def esg_predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence,(0,0,0)]
    dataset_another = [data]

    another_test = ESG_BERTDataset(dataset_another, 0, 1, word_tokenizer, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        # model의 forward 인자 설정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.float().to(device)
        # model output 도출
        out = model(token_ids, valid_length, segment_ids)
        # 모델의 output은 바로 사용하지 못한다. 밑에 있는 logits 코드 이용하기
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

    return logits


In [ ]:
def sent_predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = SENT_BERTDataset(dataset_another, 0, 1, word_tokenizer, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        # model의 forward 인자 설정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        # model output 도출
        out = model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append('e')
            elif np.argmax(logits) == 1:
                test_eval.append("s")
            elif np.argmax(logits) == 2:
                test_eval.append("g")


    return test_eval[0]

In [ ]:
# 테스트 데이터 불러오기
test_data = pd.read_csv('/content/drive/MyDrive/kobert_modeling/naver_news_test.csv')

# 문장 추출
sents = test_data['content']
for i, sent in enumerate(sents[:10]) :
    esg_output = predict(sent)
    print(f'{i+1}번 문장에 대한 esg 등급입니다.')
    print(f' E : {esg_output[0] * 100}점, S : {esg_output[1] * 100}점, G : {esg_output[2] * 100}점')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


1번 문장에 대한 esg 등급입니다.
 E : 51.00491642951965점, S : 59.80818271636963점, G : 42.548397183418274점
2번 문장에 대한 esg 등급입니다.
 E : 64.41511511802673점, S : 0.1727480790577829점, G : 99.80581402778625점
3번 문장에 대한 esg 등급입니다.
 E : 64.37705159187317점, S : 0.174202723428607점, G : 99.82020854949951점
4번 문장에 대한 esg 등급입니다.
 E : 51.73410177230835점, S : 71.17260098457336점, G : 30.541467666625977점
5번 문장에 대한 esg 등급입니다.
 E : 56.460756063461304점, S : 96.29596471786499점, G : 4.019274190068245점
6번 문장에 대한 esg 등급입니다.
 E : 64.50883746147156점, S : 0.17146732425317168점, G : 99.81313347816467점
7번 문장에 대한 esg 등급입니다.
 E : 56.77765607833862점, S : 99.8140811920166점, G : 0.18708682619035244점
8번 문장에 대한 esg 등급입니다.
 E : 56.83636665344238점, S : 99.7882068157196점, G : 0.22728776093572378점
9번 문장에 대한 esg 등급입니다.
 E : 56.57462477684021점, S : 99.80804920196533점, G : 0.18187187379226089점
10번 문장에 대한 esg 등급입니다.
 E : 64.3987774848938점, S : 0.17293673008680344점, G : 99.80529546737671점


`모델 저장하기`

In [ ]:
# 저장하기(모델 추가 계층 및 옵티마이저)
torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, '/content/drive/MyDrive/model_checkpoint/test_version1.pt')

In [ ]:
# 재적용하기
model = BERT_ESG_Classifier(kobertmodel,  dr_rate = 0.5).to(device) # 모델 및 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)

checkpoint = torch.load('/content/drive/MyDrive/model_checkpoint/test_version1.pt') # 전체 환경 load 하기

model.load_state_dict(checkpoint['model_state_dict']) # 적용하기
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 재확인하기 ---> 결과 동일
test_data = pd.read_csv('/content/drive/MyDrive/kobert_modeling/naver_news_test.csv')
# 문장만 추출
sents = test_data['content']
for i, sent in enumerate(sents[:10]) :
    esg_output = predict(sent)
    print(f'{i+1}번 문장에 대한 esg 등급입니다.')
    print(f' E : {esg_output[0] * 100}점, S : {esg_output[1] * 100}점, G : {esg_output[2] * 100}점')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


1번 문장에 대한 esg 등급입니다.
 E : 51.00491642951965점, S : 59.80818271636963점, G : 42.548397183418274점
2번 문장에 대한 esg 등급입니다.
 E : 64.41511511802673점, S : 0.1727480790577829점, G : 99.80581402778625점
3번 문장에 대한 esg 등급입니다.
 E : 64.37705159187317점, S : 0.174202723428607점, G : 99.82020854949951점
4번 문장에 대한 esg 등급입니다.
 E : 51.73410177230835점, S : 71.17260098457336점, G : 30.541467666625977점
5번 문장에 대한 esg 등급입니다.
 E : 56.460756063461304점, S : 96.29596471786499점, G : 4.019274190068245점
6번 문장에 대한 esg 등급입니다.
 E : 64.50883746147156점, S : 0.17146732425317168점, G : 99.81313347816467점
7번 문장에 대한 esg 등급입니다.
 E : 56.77765607833862점, S : 99.8140811920166점, G : 0.18708682619035244점
8번 문장에 대한 esg 등급입니다.
 E : 56.83636665344238점, S : 99.7882068157196점, G : 0.22728776093572378점
9번 문장에 대한 esg 등급입니다.
 E : 56.57462477684021점, S : 99.80804920196533점, G : 0.18187187379226089점
10번 문장에 대한 esg 등급입니다.
 E : 64.3987774848938점, S : 0.17293673008680344점, G : 99.80529546737671점


`모델2 : model + esg + 긍부정`

# 사용한 코드 설명 부분(실행하지 않는 부분)

`모델 저장하고 불러오기`
- 저장할 파라미터 대상 : 분류를 위해서 추가한 계층의 parameters [not kobert parameters]
- 저장할 때 확장자 : .pt or .pth
- 모델 저장 시 권장 방법 : torch.save(model.state_dict(), 경로)  -->  torch.save(model, 경로) 보다 가볍게 저장할 수 있음.

1. 모델의 일부 정보(state_dict())만 저장하고 불러오기 : 모델 저장 시 권장

In [ ]:
# model의 state_dict() 저장하기
torch.save(model.classifier.state_dict(), '경로 입력')

# model의 state_dict() 적용하기
model = BERT_ESG_Classifier(kobertmodel,  dr_rate = 0.5).to(device) # 1. 모델 선언
model.classifier.load_state_dict(torch.load('사전에 정의해둔 추가 계층의 파라미터의 파일을 저장한 경로')) # 2. 파라미터 적용
model.eval() # 3. 파이토치 공식문서 코멘트 : 꼭 model.eval()을 호출하여 드롭아웃 및 배치 정규화를 평가 모드로 설정하여야 합니다.

2. 전체 모델 저장하고 불러오기

In [ ]:
# 전체 모델 저장하기
torch.save(model, '경로 입력')

# 전체 모델 적용하기
model = BERT_ESG_Classifier(kobertmodel,  dr_rate = 0.5).to(device) # 1. 모델 선언
model = torch.load('저장한 경로')
model.eval()

3. 모델 + 학습 환경 저장하고 불러오기

In [ ]:
# 모델 + 학습 환경 저장하기
torch.save({'epoch': num_epochs, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss}, '경로 입력')

# 모델 + 학습 환경 적용하기
model = BERT_ESG_Classifier(kobertmodel,  dr_rate = 0.5).to(device) # 모델 및 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)

checkpoint = torch.load('저장한 경로 입력') # 전체 환경 load 하기

model.load_state_dict(checkpoint['model_state_dict']) # 적용하기
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

### 추가로 적용할 수 있는 참고 사항
4. 여러개 모델 하나의 파일에 저장하고 불러오기
5. 다른 모델의 매개변수를 사용하여 빠르게 모델 시작하기(warmstart)
6. GPU에서 저장하고 CPU에서 불러오기
7. GPU에서 저장하고 GPU에서 불러오기
8. CPU에서 저장하고 GPU에서 불러오기

`구글 코랩에서 모델 저장하고 불러오기`

In [ ]:
PATH = '/content/drive/MyDrive/Colab Notebooks/inceptionv4.pt' # 경로 입력(예시)
import os.path

epoch_start = 1

if os.path.exists(PATH): # 해당 경로에 파일이 있으면
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch_start = checkpoint['epoch'] + 1
    print("successfully loaded!")
    print("epoch saved until here: ", epoch_start-1)
    print("train starts from this epoch: Epoch ", epoch_start)

`pytorch에서 모델의 각 계층 이름 or 파라미터(가중치, 편향) 수치에 직접적으로 접근하는 방법`

In [ ]:
# torch.nn.Module.parameters() : 모델이 가지고 있는 가중치와 편향을 순서대로 보여준다.
for para in model.parameters() :
    print(para)

# torch.nn.Module.named_parameters() : 모델이 가지고 있는 (layer의 '이름', 해당 layer의 'parameter')를 순서대로 보여준다.
for name, para in model.named_parameters() :
    print(name)
    print(para)

`pytorch에서 모델의 각 계층 이름 or 파라미터(가중치, 편향) 요약 정보에 접근하는 방법`

In [ ]:
# model.children() : 계층의 특성을 요약해서 보여준다.
for child in model.children() :
    print(child)

# model.named_children() : 각 계층의 이름과 요약 정보를 보여준다
for name, child in model.named_children() :
    print(name, child)

`모델 내 존재하는 특정 계층의 파라미터를 보는 방법 - 1 : get_parameter()`

In [ ]:
# model이 가지고 있는 layer 이름 파악하기
for name, para in model.named_parameters() :
    print(name)

# 특정 layer의 파라미터 찾기
model.get_parameter('모델 내에 존재하는 특정 계층의 이름')

`모델 내 존재하는 특정 계층의 파라미터를 보는 방법 - 2 : state_dict() : {계층 이름 : 해당 계층 파라미터 값}`
- model.state_dict()
- optimizer.state_dict()

In [ ]:
# {계층이름 : 해당 계층 파라미터 값}을 가지는 객체 생성
state_dict = model.state_dict()
# 특정 계층의 파라미터 접근하는 법
state_dict['model에 존재하는 계층 이름']


`model.children() vs model.modules()`
- children : 시퀀스 단계 별 특성 정보
- module : 시퀀스 이름 + 시퀀스 단계 별 특성 정보 + 시퀀스 요약 정보

In [ ]:
# 이터레이터이고 직접적으로 보고자 할 때는 list로 바꿔야 한다
print(list(model.children()))
print(list(model.modules()))

`특정 계층만 freezing 하는 방법`
- freezing의 목적 : pre-trained model을 가지고 와서 추가 학습을 시킬 때 pre-trained 모델의 parameter가 업데이트 되는 경우 기존의 잘 학습된 특성을 잃어버릴 수 있기 때문
- freezing 방식 : pre-trained 된 부분의 파라미터는 freezing 하고 특정 task를 위해 추가로 쌓은 layer만 파라미터 업데이트를 할 수 있도록(학습할 수 있도록) 설정한다.
- freezing 판단 : 파라미터의 requires_grad(기울기 계산 할거야?)가 True : unfreezing, False : freezing  

In [ ]:
# requires_grad 접근 방법 : 1. model의 파라미터에 접근 2. model파라미터.requires_grad
for name, param in model.named_parameters():
    if name.count("fc2"): # 내가 freezing 하기를 원하는 계층이 있으면 1 이상의 숫자가 나오면서 requires_grad = False로 설정함.
        param.requires_grad = False

`model.train() vs model.eval() & with torch.no_grad()`

In [ ]:
# 학습 모드
model.train()

# 평가 모드
model.eval()
with torch.no_grad() :
    pass

`model.eval() & with torch.no_grad()이 항상 같이 등장하는 이유`
- model.eval() : 평가(테스트) 모드
- with torch.no_grad() : 아래부터는 기울기 계산을 하지 않는다.
- 항상 같이 등장하는 이유 : 평가 모드에서는 기울기를 계산할 필요가 없다. 하지만 model.eval() 모드를 작동해도 requires_grad = False 설정이 되지는 않는다. 단지  dropout, batchnorm 같은 것의 기능을 끄는 역할만 한다. 따라서 with torch.no_grad를 이용한다. 이는 requires_grad = False가 되도록 하여 불필요한 계산을 줄이는 역할을 한다.

`model.zero_grad()와 optimizer.zero_grad() 차이`
- 모델 학습 시 '한 종류'의 optimizer만 사용되는 경우 : optimizer.zero_grad() 사용 권장
- 모델 학습 시 '여러 종류'의 optimizer만 사용되는 경우 : model.zero_grad() 사용 권장
- zero_grad() 사용 이유
    - 1. loss.backward()로 Loss gradient를 역전파 한다.
    - 2. 기울기는 누적되어 저장이 된다.
    - 3. zero_grad()를 이용해서 기울기 값을 초기화시켜 이전 기울기의 영향을 제거한다.

`학습 시 일반적으로 필요한 것들`
- 1. Dataset & DataLoader
- 2. model
- 3. loss_function
- 3. optimizer
- 4. scheduler

`학습(train) 순서`
- 1. model 정의하기
- 2. loss_function 정의하기
- 3. optimizer와 scheduler 정의하기
- 4. optimizer.zero_grad() : epoch 마다 실행
- 5. model 순전파
- 6. loss 구하기
- 7. loss.backward() : 손실함수 기울기 계산
- 8. optimizer.step() / scheduler.step() : 기울기 기반 가중치 업데이트 / 학습률 조정 업데이트


`train_test_split 사용방법`

- 1. split 대상 : 리스트, 튜플, 데이터프레임
    - split의 대상이 하나(x)인 경우 : x_train, x_test 반환
    - split의 대상이 두 개(x,y)인 경우 : x_train, x_test, y_train, y_test 반환

- 2. 주요 파라미터
    -  test_size : ex) test_size = 0.2 --> 훈련 데이터 80%, 평가 데이터 20%
    - shuffle : shuffle = True(순서를 무작위로 섞기), shuffle = False(순서대로)
    - random_state = int숫자 : 매번 '동일한' '훈련 데이터'와 '평가 데이터'를 얻기 위한 설정
    - stratify : DataFrame을 split 하는 경우 --> 특정 칼럼을 지정하고 지정 칼럼의 값의 비율을 동일하게 만드는 역할(블로그 참고)